# Урок 2. Парсинг HTML. BeautifulSoup
**1. Необходимо собрать информацию о вакансиях на вводимую должность с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:**

* **наименование вакансии**
* **предлагаемую зарплату (отдельно мин. и и отдельно макс.)**
* **ссылку на саму вакансию**
* **cайт откуда собрана вакансия**

**По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.**

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
REQUEST = 'data scientist'

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)' 
                         'Chrome/78.0.3904.108 Safari/537.36'}

Сначала получим данные с сайта **Head Hunters**

In [3]:
hh_link = 'https://hh.ru/search/vacancy?area=1&L_is_autosearch=false&clusters=true&enable_snippets=true&text={}'.format(REQUEST.replace(' ', '+'))

Пишем функцию получения данных о вакансиях с конкретной страницы результатов поиска.
Отдельно вынесена функция получения данных о зарплате. Она пригодится второй раз на сайте superjob

In [4]:
def parseSalary(salary):
    sal_from = None
    sal_to = None
    currency = None 
    if salary:
        salary = re.sub(r'\s+', '', salary.text)
        if not salary.find('от'):
            re_search = re.search(r'(\d+)(.+)', salary)
            if re_search:
                sal_from = int(re_search.group(1))
                currency = re_search.group(2)
        elif not salary.find('до'):
            re_search = re.search(r'(\d+)(.+)', salary)
            if re_search:
                sal_to = int(re_search.group(1))
                currency = re_search.group(2)
        else:
            re_search = re.search(r'(\d+)-(\d+)(.+)', salary)
            if re_search:
                sal_from = int(re_search.group(1))
                sal_to = int(re_search.group(2))
                currency = re_search.group(3)
    return sal_from,  sal_to, currency
                
    
def parseHHPage(data, page_num):  
    link = hh_link + '&page=' + str(page_num) if page_num else hh_link

    html = requests.get(link, headers=HEADERS).text
    parsed_html = bs(html, 'lxml')  
    
    vacancy_list = parsed_html.findAll('div', {'data-qa': 'vacancy-serp__vacancy'})
    
    for vacancy in vacancy_list:
        vac_dict = {}
        title = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        vac_dict['Title'] = title.text
        
        salary = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        vac_dict['Salary from'], vac_dict['Salary to'], vac_dict['Currency'] = parseSalary(salary)
        
        vac_dict['Link'] = title['href']
        vac_dict['Site'] = 'hh.ru'
        
        location = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-address'})
        vac_dict['Location'] = location.text if location else None
        
        company = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
        vac_dict['Company'] = company.text if company else None
        
        data.append(vac_dict)
        
    return parsed_html

Теперь вызываем функцию для нулевой страницы. Узнаем номер последней страницы.

In [5]:
data = []
first_page_html = parseHHPage(data, None)

page_numbers = first_page_html.findAll('a', {'data-page': True})
max_page = max([int(page['data-page']) for page in page_numbers]) if page_numbers else 0

for i in range(1, max_page + 1):
    parseHHPage(data, i)

Посмотрим промежуточный итог

In [6]:
df = pd.DataFrame(data)
df

,Title,Salary from,Salary to,Currency,Link,Site,Location,Company
0,Senior Data Scientist / Главный риск-аналитик ...,220000.0,250000.0,руб.,https://hh.ru/vacancy/34664001?query=data%20sc...,hh.ru,"Москва, Нахимовский проспект","ЗЕНИТ, банк"
1,Аналитик данных / Data Scientist,120000.0,180000.0,руб.,https://hh.ru/vacancy/32377243?query=data%20sc...,hh.ru,Москва,Открытые Технологии
2,Data Scientist,NaN,NaN,None,https://hh.ru/vacancy/34870483?query=data%20sc...,hh.ru,"Москва, Савеловская",Samsung Research Center
3,ML Engineer / Data Scientist,NaN,NaN,None,https://hh.ru/vacancy/34868863?query=data%20sc...,hh.ru,"Москва, Сокол",FUSION CORE
4,"Ведущий специалист / data scientist (big data,...",150000.0,NaN,руб.,https://hh.ru/vacancy/32765739?query=data%20sc...,hh.ru,"Москва, Кузнецкий мост",ЗАО Прогностические решения
...,...,...,...,...,...,...,...,...
248,Business Analyst/Product Manager,NaN,NaN,None,https://hh.ru/vacancy/31841090?query=data%20sc...,hh.ru,Москва,ООО Перфект Арт
249,Incident Manager - GFN,NaN,NaN,None,https://hh.ru/vacancy/34750097?query=data%20sc...,hh.ru,"Москва, Марьина Роща и еще 1",NVIDIA
250,ETL-разработчик,NaN,NaN,None,https://hh.ru/vacancy/34786830?query=data%20sc...,hh.ru,"Москва, Аэропорт","Mail.Ru Group, E-commerce"
251,Ad Operations Manager (Programmatic Sales),NaN,NaN,None,https://hh.ru/vacancy/34265862?query=data%20sc...,hh.ru,Москва,AdMe


Теперь добавим в этот dataframe данные **Superjob**

Заложу механизм прохода по всем страницам. Но для нашего примера вакансий Data Scientist все умещается на одну страницу.

In [7]:
super_link = 'https://www.superjob.ru'
main_link = super_link + '/vacancy/search/?keywords={}&geo%5Bt%5D%5B0%5D=4'.format(REQUEST.replace(' ', '%20'))

Пишем функцию получения данных о вакансиях

In [8]:
def parseSJPage(data, page_num):  
    link = main_link + '&page=' + str(page_num) if page_num else main_link

    html = requests.get(link, headers=HEADERS).text
    parsed_html = bs(html, 'lxml')  
    
    vacancy_list = parsed_html.findAll('div', {'class': 'f-test-vacancy-item'})
    
    for vacancy in vacancy_list:
        vac_dict = {}
        title = vacancy.find('a', {'class': '_1QIBo'})
        vac_dict['Title'] = title.text
        salary = vacancy.find('span', {'class': '_2VHxz'})
        vac_dict['Salary from'], vac_dict['Salary to'], vac_dict['Currency'] = parseSalary(salary)
        vac_dict['Link'] = super_link + title['href']
        vac_dict['Site'] = 'superjob.ru'
        
        loc_block = vacancy.find('span', {'class': '_3Ll36'})
        if loc_block:
            loc = loc_block.find_next_sibling()
            vac_dict['Location'] = loc.text if loc else None
        else:
            vac_dict['Location'] = None

        vac_dict['Company'] = vacancy.find('a', {'class': '_205Zx'}).text
        
        data.append(vac_dict)
    return parsed_html

In [9]:
first_page_html = parseSJPage(data, None)

page_numbers = first_page_html.findAll('span', {'class': '_2GT-y'})
max_page = max([int(page.text) for page in page_numbers if page.text.isdigit()]) if page_numbers else 0
        
for i in range(1, max_page + 1):
    parseSJPage(data, i)

In [10]:
df = pd.DataFrame(data)
df

,Title,Salary from,Salary to,Currency,Link,Site,Location,Company
0,Senior Data Scientist / Главный риск-аналитик ...,220000.0,250000.0,руб.,https://hh.ru/vacancy/34664001?query=data%20sc...,hh.ru,"Москва, Нахимовский проспект","ЗЕНИТ, банк"
1,Аналитик данных / Data Scientist,120000.0,180000.0,руб.,https://hh.ru/vacancy/32377243?query=data%20sc...,hh.ru,Москва,Открытые Технологии
2,Data Scientist,NaN,NaN,None,https://hh.ru/vacancy/34870483?query=data%20sc...,hh.ru,"Москва, Савеловская",Samsung Research Center
3,ML Engineer / Data Scientist,NaN,NaN,None,https://hh.ru/vacancy/34868863?query=data%20sc...,hh.ru,"Москва, Сокол",FUSION CORE
4,"Ведущий специалист / data scientist (big data,...",150000.0,NaN,руб.,https://hh.ru/vacancy/32765739?query=data%20sc...,hh.ru,"Москва, Кузнецкий мост",ЗАО Прогностические решения
...,...,...,...,...,...,...,...,...
250,ETL-разработчик,NaN,NaN,None,https://hh.ru/vacancy/34786830?query=data%20sc...,hh.ru,"Москва, Аэропорт","Mail.Ru Group, E-commerce"
251,Ad Operations Manager (Programmatic Sales),NaN,NaN,None,https://hh.ru/vacancy/34265862?query=data%20sc...,hh.ru,Москва,AdMe
252,Архитектор BigData DWH,NaN,NaN,None,https://hh.ru/vacancy/34285564?query=data%20sc...,hh.ru,"Москва, Аэропорт","Mail.Ru Group, Решения для бизнеса"
253,Data scientist-Аналитик,NaN,NaN,None,https://www.superjob.ru/vakansii/data-scientis...,superjob.ru,Москва,НТИМИ
